In [ ]:
! pip install --quiet tqdm
! pip install --quiet dask
! pip install --quiet gcsfs

In [1]:
import pandas as pd
import os
from lib.preprocess import *
from tqdm import tnrange, tqdm_notebook
import threading 

In [2]:
def process_readmes(df, pool):
    fn = Preprocessor(readme_processor, 1).process
    df = df[df.content.notna()].reset_index()
    processed = pool.map(fn, df.content)
    df['content'] = processed
    df = df[df.content.notna()].reset_index()
    df['repo_owner'] = df.repo_name.map(lambda s: s.split('/')[0])
    return df

In [3]:
def process_claims(df, pool):
    key = 'description_text'
    fn = Preprocessor(claims_processor, 4).process
    df = df[df[key].notna()].reset_index()
    processed = pool.map(fn, df[key])
    df['content'] = processed
    df = df[df.content.notna()].reset_index()
    return df

In [8]:
from  multiprocessing import Pool

In [11]:
get_files = lambda inpath: list(os.walk(inpath))[0][2]

def preprocess(outfile, inpath, fn, id_key, compression=None):
    pool = Pool()
    files = get_files(inpath)
    dfs = (pd.read_csv('{}/{}'.format(inpath, f), compression=compression) 
           for f in files)
    with open(outfile, 'w') as f:
        for df in tqdm_notebook(dfs, total = len(files)):
            df = fn(df, pool)
            for i,c in zip(df[id_key], df.content):
                f.write('{},{}\n'.format(i,c))
    pool.close()
    pool.join()

In [12]:
preprocess('claims_lookup.csv',  'oss_bigquery_exports/patent-descriptions', 
                                                   process_claims, 
                                                   'application_number_formatted', 'gzip')

Process ForkPoolWorker-141:
Process ForkPoolWorker-158:
Process ForkPoolWorker-155:
Process ForkPoolWorker-131:
Process ForkPoolWorker-151:
Process ForkPoolWorker-139:
Process ForkPoolWorker-146:
Process ForkPoolWorker-149:
Process ForkPoolWorker-147:
Process ForkPoolWorker-132:
Process ForkPoolWorker-140:
Process ForkPoolWorker-143:
Process ForkPoolWorker-159:
Process ForkPoolWorker-153:
Process ForkPoolWorker-134:
Process ForkPoolWorker-130:
Process ForkPoolWorker-154:
Process ForkPoolWorker-152:
Process ForkPoolWorker-160:
Process ForkPoolWorker-138:
Process ForkPoolWorker-135:
Process ForkPoolWorker-137:
Process ForkPoolWorker-157:
Process ForkPoolWorker-145:
Process ForkPoolWorker-144:
Process ForkPoolWorker-136:
Process ForkPoolWorker-133:
Process ForkPoolWorker-148:
Process ForkPoolWorker-150:
Process ForkPoolWorker-129:
Process ForkPoolWorker-156:
Process ForkPoolWorker-142:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

## Run for different datas

In [ ]:
preprocess('readme_lookup.csv', 'oss_bigquery_exports/readmes', process_readmes, 'id')

### Treat owner like collaborative filtering? 

In [101]:
by_owner = df.groupby('repo_owner').apply(lambda df: ' \t '.join(df.content))

In [102]:
with open('by_owner.txt', 'w') as f:
    for c in by_owner:
        f.write(c + '\n')